In [1]:
%pip install gensim
%pip install torch
%pip install Pillow
%pip install requests
%pip install open_clip_torch
%pip install torchvision


  Using cached gensim-4.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (8.3 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.4 kB)
Using cached gensim-4.3.3-cp39-cp39-macosx_11_0_arm64.whl (24.0 MB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl (30.3 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl (38 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached torch-2.6.0-cp39-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gensim.downloader as api # pip install gensim
from txt2png import txt_to_png
from open_clip_test_mps import clip_loss, device, preprocess

In [19]:
word_model_name = "glove-twitter-25"
word_model = api.load(word_model_name)
embedding_dim = 25
x_dim = 25
y_dim = 8
output_size = y_dim * x_dim
ascii_chars = " .,:;+*#@$%&0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-_=|/\\()[]{}"
num_chars = len(ascii_chars)
char_to_index = {char: i for i, char in enumerate(ascii_chars)}
index_to_char = {i: char for i, char in enumerate(ascii_chars)}

class ASCIIArtGenerator(nn.Module):
    def __init__(self, input_size, output_size, num_chars):
        super(ASCIIArtGenerator, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size * num_chars)
        self.num_chars = num_chars
        self.output_size = output_size

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x.view(-1, self.output_size, self.num_chars)

def generate_ascii_art(model, embedding, temperature=1.0):
    model.eval()
    with torch.no_grad():
        output = model(embedding.unsqueeze(0))  # Shape: [1, output_size, num_chars]
        
        # Apply temperature scaling to logits
        scaled_logits = output / temperature
        
        # Convert to probabilities
        probs = F.softmax(scaled_logits, dim=2)
        
        # Sample from the distribution
        # We'll use multinomial sampling which selects based on probabilities
        indices = torch.multinomial(probs.view(-1, num_chars), 1).squeeze(-1)
        
        ascii_grid = ""
        for i in range(y_dim):
            for j in range(x_dim):
                ascii_grid += index_to_char[indices[i * x_dim + j].item()]
            ascii_grid += "\n"
        return ascii_grid

model = ASCIIArtGenerator(embedding_dim, output_size, num_chars)

In [ ]:
embedding = torch.tensor(word_model["cat"])
optimizer = optim.Adam(model.parameters(), lr = .01)
for i in range(10):
    ascii_art = generate_ascii_art(model, embedding)
    ascii_png = txt_to_png(ascii_art, output_png="temp.png", save=True)
    image_tensor = preprocess(ascii_png).unsqueeze(0).to(device, dtype=torch.float32)
    optimizer.zero_grad()
    candidate_labels = ["bird", "dog", "cat", "castle"]
    true_label = 2
    loss = clip_loss(image_tensor, candidate_labels, true_label)
    loss.backward()
    optimizer.step()
    print(f"Iteration {i}, Loss: {loss.item()}")
    print(ascii_art)
    print()

Iteration 0, Loss: 0.5660963654518127
O|V&/dc[*4t(9,zkZkwp7H=Q_
Bbh1wM(.;1):I/D,E(|Sf]m@|
WmGwVG4RQD_y;bM]d&JkY;-j5
5P -bmC6&;g IHhY:2JxHETp$
\0EzJ.m8]p;dc-[pogRBb-B=$
9FuW=YD2lAnjdJ$+uFl(ehfFx
%&)kK-=#{{Ppncy$,(Qkyrws:
3W[m*fkzU[&pmf5G#\wm}@\ :


Iteration 1, Loss: 1.252697467803955
$C.@2:8&g${N/R$nl+BJpbOg{
\s-Oy9EaclxKBaL8UH{+2RFd#
My\AGt2-mwtE)l3/}_hur2JhH
#Y;,l|8X&-k\.C=y;xu{LeW=n
]djr%sSYsNx,a#zUcj:xnSfqI
uKN=YAWxh[SXBx&}WSFCz]DaS
qyjGQtVgoVBhY0p2lx@70Y%OH
]XAH3PfO p-}%]s;2\h|2zq{,


Iteration 2, Loss: 0.7567050457000732
SEK +Zw|Y;[QwOV]a,H0KNsU[
Z/vb|A7.5dx@xqjUvk %Y(Ale
e}o bSF97cdWBKgz+BxJy\__$
-)fzC6$H+i%73fVYbi3iuzkB[
}(ef7Rc,/xnm9@sRqaH#I:vO(
frWgP)&CnEBOz*7KukxD :0%:
u+,bIo,Y2kPIuqfDs+B}aI y7
$*cS)ru2e=PV[M]_E8ut_wzNP


Iteration 3, Loss: 0.4970836937427521
4SJTl)hkcPDUfeH8EoFy{} #2
EOH-SISp#lNEPyG5XO$HqvN1P
t0own5r0xRU;:1.F|,XsCQMC4
,Pg6Tf98k)eWbY\6)wgR}EVA:
0h/(Nc1VCr*HyWP6kUYGz7+Lh
)Aa=I0uLgm[eGHyU:*yMDNcS6
hsw9ulVU1TjmM)}GQqh6D(xcn
FNY1fi3JGCJ:LCDVP$9-9WX-G


Iteration